refer to [2023-03-08_logbook](/Users/jonathan/001_obsidian_vault/2023-03-08_logbook.md)

# Investigating Wavelength of Absorbance Maxima Outside of Mobile Phase Region of Wines

The purpose of this notebook is to:

- [x] Isolate the sequences and runs which used the avantor column.
- [ ] Identify the wavelength region of 'minimal' baseline.
- [ ] Identify the single wavelength with maximal absorbance within that region.

First I need to set up the environment.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

import os

# adds root dir 'wine_analyis_hplc_uv' to path.

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

from agilette import agilette_core as ag

lib = ag.Agilette('/Users/jonathan/0_jono_data').library

In [2]:
lib_df = lib.data_table().applymap(lambda x: x.lower() if type(x) == str else x)
lib_df.head()

,names,path,sequence,ch_files,uv_files,method,desc
2023-03-01 17:22:18,acetone0006,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 17:02:51,acetone0005,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 16:43:21,acetone0004,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 16:23:51,acetone0003,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 16:04:20,acetone0002,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty


In [3]:
lib_df.describe()

,names,path,sequence,ch_files,uv_files,method,desc
count,103,103,103,103,103,103,103
unique,69,103,8,4,2,16,10
top,z3,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,single run,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
freq,12,1,39,47,62,32,86


In [4]:
avantor_runs = lib_df[lib_df['method'].str.contains('avantor')]
avantor_runs.head()

,names,path,sequence,ch_files,uv_files,method,desc
2023-03-01 17:22:18,acetone0006,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 17:02:51,acetone0005,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 16:43:21,acetone0004,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 16:23:51,acetone0003,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
2023-03-01 16:04:20,acetone0002,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,2023-03-01_15-22-02_acetone_void-vol-measureme...,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty


In [5]:
avantor_runs.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 63 entries, 2023-03-01 17:22:18 to 2023-02-07 12:01:43
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   names     63 non-null     object
 1   path      63 non-null     object
 2   sequence  63 non-null     object
 3   ch_files  63 non-null     object
 4   uv_files  63 non-null     object
 5   method    63 non-null     object
 6   desc      63 non-null     object
dtypes: object(7)
memory usage: 3.9+ KB


In [6]:
avantor_runs.describe()

,names,path,sequence,ch_files,uv_files,method,desc
count,63,63,63,63,63,63,63
unique,42,63,5,2,2,5,9
top,z3,/Users/jonathan/0_jono_data/2023-03-01_15-22-0...,single run,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[DAD1.UV],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty
freq,3,1,18,43,37,32,50


Looking good so far! 63 runs to play with, apparently only 42 are unique. Interesting. What does that mean?

In [7]:
avantor_runs_dup_df = avantor_runs[avantor_runs.duplicated(subset = ['names'], keep = False)]
avantor_runs_dup_df.head()

,names,path,sequence,ch_files,uv_files,method,desc
2023-02-23 12:21:12,2021-debortoli-cabernet-merlot_avantor,/Users/jonathan/0_jono_data/2023-02-23_2021-DE...,single run,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[DAD1.UV],c:\chem32\1\methods\0_jono_methods\avantor100x...,avantor-150-x-4.6-c18-h2o-meoh-2.1%--gradient
2023-02-23 11:25:03,lor-ristretto,/Users/jonathan/0_jono_data/2023-02-23_LOR-RIS...,single run,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[DAD1.UV],c:\chem32\1\methods\0_jono_methods\avantor100x...,avantor-150-x-4.6-c18-h2o-meoh-2.1%--gradientp...
2023-02-22 17:39:06,2021-debortoli-cabernet-merlot_avantor,/Users/jonathan/0_jono_data/2023-02-22_2021-DE...,single run,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[DAD1.UV],c:\chem32\1\methods\0_jono_methods\avantor100x...,halo-150-x-4.6-c18-h2o-meoh-2.1%--gradient
2023-02-22 09:48:25,lor-ristretto,/Users/jonathan/0_jono_data/2023-02-22_LOR-RIS...,single run,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[DAD1.UV],c:\chem32\1\methods\0_jono_methods\avantor100x...,avantor-100-x-4.6-c18-h2o-meoh-2.1%--gradient
2023-02-16 04:45:45,0211,/Users/jonathan/0_jono_data/2023-02-15_WINES_2...,2023-02-15_wines_2023-02-15_15-19-53.sequence,"[DAD1D.ch, DAD1E.ch, DAD1A.ch, DAD1F.ch, DAD1B...",[DAD1.UV],c:\chem32\1\methods\0_jono_methods\avantor100x...,empty


Duplicates in name column are occuring simply because I'm running the same sample multiple times.

Now, there is a question of baseline analysis. Let's begin as usual by lecting one data set, creating a process for that one, then generalising to all. Let's do 2021-debortoli-cabernet-merlot_avantor. First q: how do we interface between the data table and the rest of Agilette? feed the pathname back in?

In [8]:
the_run = avantor_runs[(avantor_runs['names'] == "2021-debortoli-cabernet-merlot_avantor") & (avantor_runs.index == '2023-02-23 12:21:12')]['path']

the_run.values
#lib.single_runs[the_run]

array([PosixPath('/Users/jonathan/0_jono_data/2023-02-23_2021-DEBORTOLI-CABERNET-MERLOT_AVANTOR.D')],
      dtype=object)

In [9]:
wine_run_dir = lib.single_runs[the_run.values[0].name]
help(wine_run_dir)

Help on Run_Dir in module agilette.agilette_core object:

class Run_Dir(builtins.object)
 |  Run_Dir(file_path)
 |  
 |  A single run directory containing signal data and metadata about the run.
 |  
 |  Use extract_uv_data(self) to get the Spectrum.
 |  
 |  To get ch data, it is useful to view each signal's wavelength and band rather than the detector name, so run self.extract_ch_data() to get a dictionary of each signal with its wavelength as the key and dataframe of the signal as value.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, file_path)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  data_files_dicter(self)
 |  
 |  extract_ch_data(self)
 |      A function to extraact the ch data from the directory. It was necessary to implement this as a function to provide a 'switch' to parse the data, as that is a slow process.
 |      
 |      Calling this function returns a dict whose keys are the s

In [12]:
wine_uv_data = wine_run_dir.get_uv_data()
wine_uv_data.extract_uv_data()

uv_data extracted


In [14]:
wine_uv_data.uv_data.head()

,190,192,194,196,198,200,202,204,206,208,...,382,384,386,388,390,392,394,396,398,400
mins,,,,,,,,,,,,,,,,,,,,,
0.000200,2.107166,1.578845,1.064405,0.535600,0.153184,-0.037678,-0.112697,-0.154629,-0.175335,-0.164993,...,-0.021599,-0.026613,-0.028759,-0.029624,-0.029638,-0.031993,-0.030324,-0.026256,-0.022732,-0.026464
0.006867,2.294607,1.698248,1.130253,0.563681,0.158451,-0.043951,-0.123017,-0.166535,-0.187978,-0.178419,...,-0.033312,-0.038251,-0.039652,-0.040576,-0.041284,-0.043266,-0.041336,-0.036664,-0.033580,-0.037007
0.013533,2.442166,1.793459,1.183547,0.587337,0.163995,-0.047535,-0.129640,-0.174522,-0.196412,-0.187300,...,-0.042044,-0.047050,-0.047661,-0.048392,-0.049658,-0.050955,-0.048682,-0.043869,-0.040710,-0.044085
0.020200,2.519920,1.845367,1.213342,0.601672,0.169002,-0.047378,-0.130855,-0.176273,-0.198305,-0.189617,...,-0.045516,-0.050701,-0.050783,-0.051044,-0.052564,-0.052929,-0.050649,-0.046305,-0.042498,-0.046037
0.026867,2.507798,1.841180,1.212925,0.603572,0.172310,-0.043452,-0.126049,-0.170939,-0.192836,-0.184655,...,-0.043266,-0.048503,-0.047974,-0.047535,-0.049144,-0.048555,-0.046641,-0.043206,-0.038326,-0.042178
